In [64]:
import mxnet as mx

# SVD exapmle

In [65]:
import numpy as np
data  = np.array([[1,2,3],[2,0,3],[3,3,0]])
U,S,V = np.linalg.svd(data)

In [66]:
U

array([[-0.60552477, -0.29936989, -0.73737197],
       [-0.52683681, -0.54365974,  0.6533583 ],
       [-0.59647525,  0.78409933,  0.17148036]])

In [67]:
A = np.dot(U, np.diag(S))
A

array([[-3.44862415, -0.96560861, -1.08378572],
       [-3.00047531, -1.7535582 ,  0.96030284],
       [-3.39708474,  2.52908889,  0.252041  ]])

In [68]:
B = np.dot(A, V)
B

array([[  1.00000000e+00,   2.00000000e+00,   3.00000000e+00],
       [  2.00000000e+00,   1.22542781e-15,   3.00000000e+00],
       [  3.00000000e+00,   3.00000000e+00,  -1.73777855e-16]])

# mxnet 实现SVD

def plain_net(max_user, max_item, k):
    # input
    user = mx.symbol.Variable('user')
    item = mx.symbol.Variable('item')
    score = mx.symbol.Variable('score')
    # user feature lookup
    user = mx.symbol.Embedding(data = user, input_dim = max_user, output_dim=k)
    # item feature lookup
    item = mx.symbol.Embedding(data = item, input_dim = max_item, output_dim=k)
    # predict by the inner product, which is elementwise product and then sum
    pred = user * item
    pred = mx.symbol.sum(data = pred, axis = 1)
    pred = mx.symbol.Flatten(data = pred)
    # loss layer
    pred = mx.symbol.LinearRegressionOutput(data = pred, label = score)
    return pred


In [69]:
def my_line_svd():
    user = mx.symbol.Variable('user')
    item = mx.symbol.Variable('item')
    score = mx.symbol.Variable('score')
    user_embedding = mx.symbol.Embedding(data=user, input_dim=3, output_dim=3)
    item_embedding = mx.symbol.Embedding(data=item, input_dim=3, output_dim=3)
    pred = mx.symbol.sum(user_embedding * item_embedding, axis=1)
    pred = mx.symbol.Flatten(data=pred)
    pred = mx.symbol.LinearRegressionOutput(data=pred, label=score)
    return pred

In [84]:

new_data = np.array([
    [0, 0],
    [0, 1],
    [0, 2],
    [1, 0],
    [1, 1],
    [1, 2],
    [2, 0],
    [2, 1],
    [2, 2]
])
label_data = np.array([1, 2, 3, 2, 0, 3, 3, 3, 0])

In [85]:
new_data[:, 0]

array([0, 0, 0, 1, 1, 1, 2, 2, 2])

In [86]:
data = {'user': new_data[:, 0],  'item': new_data[:, 1]}

data_iter = mx.io.NDArrayIter(data=data, 
                                   label=label_data, 
                                   batch_size=7, 
                                   shuffle=True, 
                                   label_name='score')

In [87]:
net=my_line_svd()
model = mx.mod.Module(
    symbol=net,
    data_names=['user', 'item'],
    label_names=['score']
)

In [88]:
mx.viz.plot_network(symbol=net)#.view()

In [89]:
model.fit(train_data=data_iter,
          optimizer_params={'learning_rate':0.5, 'momentum': 0.9},
          num_epoch=50,
          eval_metric='mse')

In [90]:
model.get_params()[0]

{'embedding6_weight': 
 [[ 0.15002407  1.85885751  0.01497641]
  [ 1.4209739   0.55653608  0.28029177]
  [-0.20259452  1.03476465  2.13978648]]
 <NDArray 3x3 @cpu(0)>, 'embedding7_weight': 
 [[ 0.9968161   0.43965358  1.28818595]
  [-0.58748448  1.12861419  0.80485594]
  [ 1.62647915  1.49862492 -0.57158279]]
 <NDArray 3x3 @cpu(0)>}

In [92]:
A = model.get_params()[0]["embedding6_weight"]
B = model.get_params()[0]["embedding7_weight"]

In [93]:
mx.nd.dot(A, B.transpose())


[[  9.86092210e-01   2.02185011e+00   3.02118111e+00]
 [  2.02220058e+00   1.89088751e-02   2.98501325e+00]
 [  3.00943136e+00   3.00909114e+00  -1.85680226e-03]]
<NDArray 3x3 @cpu(0)>

## more example see: 
## https://github.com/apache/incubator-mxnet/tree/master/example/recommenders